<a href="https://colab.research.google.com/github/fjadidi2001/AD_Prediction/blob/main/HippoWithResUNetandU_net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Cell 1: Setup and Imports
# Purpose: Import libraries, mount Google Drive, and set random seed.
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import ResNet50
import cv2
import zipfile
from google.colab import drive
drive.mount('/content/drive')
seed = 42
np.random.seed(seed)
tf.random.set_seed(seed)
print(f"TensorFlow version: {tf.__version__}")
print("Environment setup complete.")

Mounted at /content/drive
TensorFlow version: 2.18.0
Environment setup complete.


# Define Metrics

In [4]:
# Purpose: Define custom metrics for segmentation evaluation (Dice, IoU, precision, recall).
def dice_coef(y_true, y_pred, smooth=1e-6):
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + smooth)

def iou(y_true, y_pred, smooth=1e-6):
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    union = tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) - intersection
    return (intersection + smooth) / (union + smooth)

def compute_metrics(y_true, y_pred, threshold=0.5):
    y_pred_bin = (y_pred > threshold).astype(np.float32)
    dice = dice_coef(y_true, y_pred_bin).numpy()
    iou_score = iou(y_true, y_pred_bin).numpy()
    y_true_f = y_true.flatten()
    y_pred_f = y_pred_bin.flatten()
    precision = np.sum(y_true_f * y_pred_f) / (np.sum(y_pred_f) + 1e-6)
    recall = np.sum(y_true_f * y_pred_f) / (np.sum(y_true_f) + 1e-6)
    return dice, iou_score, precision, recall

print("Metrics defined: Dice, IoU, Precision, Recall")

Metrics defined: Dice, IoU, Precision, Recall


# Data Loading

In [14]:
# Cell 3: Data Loading
# Purpose: Load and preprocess hippocampus MRI images and masks from zip file.
def load_image(img_path, target_size=(128, 128)):
    img = cv2.imread(img_path, cv2.IMREAD_COLOR)
    if img is None:
        raise ValueError(f"Failed to load image: {img_path}")
    img = cv2.resize(img, target_size)
    img = img / 255.0
    return img.astype(np.float32)

def load_mask(mask_path, target_size=(128, 128)):
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    if mask is None:
        raise ValueError(f"Failed to load mask: {mask_path}")
    mask = cv2.resize(mask, target_size)
    mask = (mask > 0).astype(np.float32)
    mask = mask[:, :, np.newaxis]
    return mask

# Unzip dataset
zip_path = '/content/hippocampus segmentation dataset.zip'
extract_path = '/content/hippocampus_dataset'
if not os.path.exists(zip_path):
    raise FileNotFoundError(f"Zip file not found: {zip_path}")
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Recursive file search
def find_files(directory, extensions):
    files = []
    for root, _, filenames in os.walk(directory):
        for f in filenames:
            if any(f.lower().endswith(ext) for ext in extensions):
                files.append(os.path.join(root, f))
    return sorted(files)

# Find images and masks
image_extensions = ('.jpg', '.jpeg', '.png')
mask_extensions = ('.jpg', '.jpeg', '.png')
all_files = find_files(extract_path, image_extensions + mask_extensions)
image_files = [f for f in all_files if not any(kw in os.path.basename(f).lower() for kw in ['mask', '_mask', 'seg', 'segmentation'])]
mask_files = [f for f in all_files if any(kw in os.path.basename(f).lower() for kw in ['mask', '_mask', 'seg', 'segmentation'])]

# Debug prints
print(f"Found {len(image_files)} image files, {len(mask_files)} mask files")
if image_files:
    print("Sample images:", image_files[:3])
if mask_files:
    print("Sample masks:", mask_files[:3])

# Pair images and masks
train_data = []
if len(image_files) == len(mask_files):
    # Try index-based pairing if equal number of files
    print("Attempting index-based pairing (sorted files)")
    train_data = list(zip(image_files, mask_files))
else:
    # Flexible name-based pairing
    print("Attempting name-based pairing")
    for img in image_files:
        img_name = os.path.basename(img).lower().split('.')[0]
        for mask in mask_files:
            mask_name = os.path.basename(mask).lower().split('.')[0]
            # Match if names share a common identifier
            if img_name.replace('img', '') in mask_name or mask_name.replace('mask', '').replace('seg', '') in img_name:
                train_data.append((img, mask))
                break
            # Alternative: Replace 'img' with 'mask' or 'seg'
            if img_name.replace('img', 'mask') == mask_name or img_name.replace('img', 'seg') == mask_name:
                train_data.append((img, mask))
                break

print(f"Paired {len(train_data)} image-mask pairs")
if not train_data:
    # Debug unmatched files
    print("Unmatched images:", [os.path.basename(f) for f in image_files[:5]])
    print("Unmatched masks:", [os.path.basename(f) for f in mask_files[:5]])
    raise ValueError("No image-mask pairs found. Check file names or directory structure. See unmatched files above.")

# Load data
X = np.array([load_image(img_path) for img_path, _ in train_data])
Y = np.array([load_mask(mask_path) for _, mask_path in train_data])

# Split data
if X.shape[0] == 0:
    raise ValueError("No data loaded. Cannot proceed with empty dataset.")
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, random_state=seed)
print(f"X_train shape: {X_train.shape}, X_test shape: {X_test.shape}")
print(f"Y_train shape: {Y_train.shape}, Y_test shape: {Y_test.shape}")

Found 499 image files, 998 mask files
Sample images: ['/content/hippocampus_dataset/aug/images/img_aug1.jpg', '/content/hippocampus_dataset/aug/images/img_aug10.jpg', '/content/hippocampus_dataset/aug/images/img_aug100.jpg']
Sample masks: ['/content/hippocampus_dataset/aug/masks/left/maskleft_aug1.jpg', '/content/hippocampus_dataset/aug/masks/left/maskleft_aug10.jpg', '/content/hippocampus_dataset/aug/masks/left/maskleft_aug100.jpg']
Attempting name-based pairing
Paired 499 image-mask pairs
X_train shape: (449, 128, 128, 3), X_test shape: (50, 128, 128, 3)
Y_train shape: (449, 128, 128, 1), Y_test shape: (50, 128, 128, 1)
